In [29]:
# !pip install numpy
# !pip install opencv-python
# !pip install matplotlib
# !pip install transformers
# !pip install requests
# !pip install torchvision
# !pip install Pillow
# !pip install git+https://github.com/facebookresearch/segment-anything.git

In [30]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2


def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x["area"]), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann["segmentation"]
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:, :, i] = color_mask[i]
        ax.imshow(np.dstack((img, m * 0.35)))


import sys

sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

# device = "cuda"
device = torch.device("cpu")


sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)


## Drake code starts

In [31]:
import numpy as np
from pydrake.all import (
    AbstractValue,
    AngleAxis,
    Concatenate,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.icp import IterativeClosestPoint
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.mustard_depth_camera_example import MustardPointCloud
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
from manipulation.scenarios import *
from manipulation.scenarios import AddIiwaDifferentialIK, MakeManipulationStation

import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pydot
import CrackerBoxPointCloud
from IPython.display import SVG, clear_output, display
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    AngleAxis,
    DiagramBuilder,
    FindResourceOrThrow,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MeshcatVisualizer,
    SnoptSolver,
    Solve,
    eq,
    ge,
    le,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    Quaternion,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.all import (
    AbstractValue,
    AngleAxis,
    Concatenate,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook, FindResource
from manipulation.scenarios import (
    AddIiwaDifferentialIK,
    MakeManipulationStation,
    AddMultibodyTriad,
    AddShape,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
)
from pydrake.manipulation.planner import DoDifferentialInverseKinematics
from manipulation.icp import IterativeClosestPoint
from pydrake.all import (
    Box,
    Sphere,
    PrismaticJoint,
    RevoluteJoint,
    SpatialInertia,
    UnitInertia,
    BaseField,
    Fields,
)  # PlanarJoint
from datetime import datetime

if running_as_notebook:
    mpld3.enable_notebook()

from PIL import Image


def saat(msg=""):
    print(str(datetime.now()), msg)


saat()

2023-05-31 16:14:17.430788 


In [32]:
import numpy as np
from pydrake.all import (
    AbstractValue,
    AngleAxis,
    Concatenate,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.icp import IterativeClosestPoint
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.mustard_depth_camera_example import MustardPointCloud
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
from manipulation.scenarios import *
from manipulation.scenarios import AddIiwaDifferentialIK, MakeManipulationStation

import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pydot
import CrackerBoxPointCloud
from IPython.display import SVG, clear_output, display
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    AngleAxis,
    DiagramBuilder,
    FindResourceOrThrow,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MeshcatVisualizer,
    SnoptSolver,
    Solve,
    eq,
    ge,
    le,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    Quaternion,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    TrajectorySource,
)
from pydrake.all import (
    AbstractValue,
    AngleAxis,
    Concatenate,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook, FindResource
from manipulation.scenarios import (
    AddIiwaDifferentialIK,
    MakeManipulationStation,
    AddMultibodyTriad,
    AddShape,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
)
from pydrake.manipulation.planner import DoDifferentialInverseKinematics
from manipulation.icp import IterativeClosestPoint
from pydrake.all import (
    Box,
    Sphere,
    PrismaticJoint,
    RevoluteJoint,
    SpatialInertia,
    UnitInertia,
)  # PlanarJoint
from datetime import datetime

if running_as_notebook:
    mpld3.enable_notebook()

if True:
    global masks_saved


def saat(msg=""):
    print(str(datetime.now()), msg)


saat()

2023-05-31 16:14:17.545837 


In [33]:
from pydrake.systems.sensors import (
    CameraInfo,
    ImageDepth32F,
    ImageLabel16I,
    ImageRgba8U,
    RgbdSensor,
)

# Meshcat

In [34]:
# print(str(datetime.now()))
# Start the visualizer.

meshcat = StartMeshcat()

saat("meshcat started")


INFO:drake:Meshcat listening for connections at http://localhost:7001


2023-05-31 16:14:17.717111 meshcat started


# directives 

In [35]:
model_directives = """
directives:
- add_model:
    name: iiwa
    file: file://code/code/iiwa_description/iiwa7/iiwa7_mor_siyah.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_no_collision.sdf
        
    # file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf 

    default_joint_positions:
        iiwa_joint_0_x: [-1.0]
        iiwa_joint_0_y: [1.0]
        iiwa_joint_0_z: [0.0]
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0, -0.1, 0.0]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, -0., 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

- add_model:
    name: storage
    file: file://code/code/iiwa_description/iiwa7/storage.sdf
    # file: file://root/work/iiwa_description/iiwa7/storage.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_no_collision.sdf
        
    # file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf 

    default_joint_positions:
        iiwa_joint_0_x: [-1.0]
        iiwa_joint_0_y: [0]
        iiwa_joint_0_z: [0.0]

- add_weld:
    parent: world
    child: storage::iiwa_link_0
    X_PC:
        translation: [0, 0.0, 0.0]
        rotation: !Rpy { deg: [0, 0, 0]}


# add second iiwa
- add_model:
    name: iiwa2
    file: file://code/code/iiwa_description/iiwa7/iiwa7_mor_siyah.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf
    # file: file://root/work/iiwa_description/iiwa7/iiwa7_no_collision.sdf
        
    # file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf 

    default_joint_positions:
        iiwa_joint_0_x: [5.0]
        iiwa_joint_0_y: [5.0]
        iiwa_joint_0_z: [0.0]
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa2::iiwa_link_0
    X_PC:
        translation: [0, -0.1, 0.0]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: wsg2
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa2::iiwa_link_7
    child: wsg2::body
    X_PC:
        translation: [0, -0., 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

# - add_frame:
#     name: bin0_origin
#     X_PF:
#       base_frame: world
#       rotation: !Rpy { deg: [0.0, 70.0, 90.0 ]}
#       translation: [-0.1, -1.1, 0.45]

# - add_model:
#     name: bin0
#     # file: package://drake/examples/manipulation_station/models/bin.sdf
#     file: package://manipulation/shelves.sdf
# - add_weld:
#     parent: bin0_origin
#     child: bin0::bin_base
- add_frame:
    name: shelf0_goal
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0, 90.0 ]}
      translation: [-0.1, -1.0, 0.55]
    #   translation: [0.95, -0.3, 0.48]

- add_model:
    name: shelf0
    file: file://code/code/custom_shelf.sdf

- add_weld:
    parent: shelf0_goal
    child: shelf0::shelves_body


- add_frame:
    name: shelf2_goal
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0, 90.0 ]}
      translation: [0.5, -1.0, 0.55]
    #   translation: [0.95, -0.3, 0.48]

- add_model:
    name: shelf2
    file: file://code/code/custom_shelf.sdf

- add_weld:
    parent: shelf2_goal
    child: shelf2::shelves_body

- add_frame:
    name: shelf3_goal
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0, 90.0 ]}
      translation: [4.55, 2.97, 0.58]
    #   translation: [0.95, -0.3, 0.48]

- add_model:
    name: shelf3
    file: file://code/code/custom_shelf.sdf

- add_weld:
    parent: shelf3_goal
    child: shelf3::shelves_body 

- add_frame:
    name: shelf1_goal
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0, 90.0 ]}
      translation: [3.95, 2.97, 0.58]
    #   translation: [0.95, -0.3, 0.48]

- add_model:
    name: shelf1
    file: file://code/code/custom_shelf.sdf

- add_weld:
    parent: shelf1_goal
    child: shelf1::shelves_body


- add_model:
    name: floor
    file: package://manipulation/floor.sdf

- add_weld:
    parent: world
    child: floor::box
    X_PC:
        translation: [0, 0, -.5]

# cracker box
# add cracker box
# - add_model:
#     name: cracker_box
#     file: package://drake/manipulation/models/ycb/sdf/003_cracker_box.sdf
#     default_free_body_pose:
#         base_link_cracker:
#             translation: [-0.2, -0.95, 0.45]
#             rotation: !Rpy { deg: [-90, 0, 90] }
# - add_model:
#     name: cracker_box_2
#     file: package://drake/manipulation/models/ycb/sdf/003_cracker_box.sdf
#     default_free_body_pose:
#         base_link_cracker:
#             translation: [+0.0, -0.95, 0.45]
#             rotation: !Rpy { deg: [-90, 0, 90] }

# add cameras    
- add_frame:
    name: camera4_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [70, 180, 0]}
        translation: [-0.05, -.3, .7]

- add_model:
    name: camera4
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera4_origin
    child: camera4::base

- add_frame:
    name: camera5_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [70, 180, 0]}
        translation: [0.55, -.3, .7]

- add_model:
    name: camera5
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera5_origin
    child: camera5::base

"""

saat("model directives defined")

2023-05-31 16:14:17.808790 model directives defined


In [36]:
import pickle


class MaskSystem(LeafSystem):
    def __init__(self, rgbd_sensor):
        LeafSystem.__init__(self)
        self.DeclareAbstractInputPort(
            "rgb_image", rgbd_sensor.color_image_output_port().Allocate()
        )
        self.DeclareAbstractInputPort(
            "depth_image", rgbd_sensor.depth_image_32F_output_port().Allocate()
        )

        self.DeclareAbstractOutputPort(
            "masked_depth_image",
            lambda: AbstractValue.Make([np.ndarray]),
            self.get_masks,
        )
        self.color_image = None
        self.depth_image = None

    def get_masks(self, context, output):
        thresh = 0.97
        # print(self.EvalAbstractInput(context, 'rgb_image').get_value())
        # res = np.repeat(rgb_image[np.newaxis, :, :], 5, axis=0)

        self.color_image = self.GetInputPort("rgb_image").Eval(context).data
        self.color_image = Image.fromarray(np.uint8(self.color_image)).convert("RGB")
        self.depth_image = self.GetInputPort("depth_image").Eval(context).data
        self.depth_image = self.depth_image.squeeze()

        # save color image with timestamp info added into folder named color images
        self.color_image.save("color_image/color_image.png" + str(datetime.now()) + ".png")

        # convert color_image to np array
        self.color_image = np.array(self.color_image)

        # convert depth_image to np array
        self.depth_image = np.array(self.depth_image)

        rgbImages = []
        masked_depth_images = []
        predicted_pieces = []

        masks = mask_generator.generate(self.color_image)

        print("before pcikle")
        # save variable masks into a file using pickle
        with open('masks.pkl', 'wb') as f:
            pickle.dump(masks, f)

        print("after save")
        # load variable masks from a file using pickle
        # with open("masks.pkl", "rb") as f:
        #     masks = pickle.load(f)
        print("after pickle")

        for i in range(len(masks)):

            # !!! SELECTING THE MASKS. HARCODED VALUES MAY DEPEN ON THE POSITION OF THE CAMERA !!!s
            selected = (
                masks[i]["area"] >= 9000
                and masks[i]["area"] <= 20000
                and masks[i]["bbox"][2] >= 60
                and masks[i]["bbox"][2] <= 120
                and masks[i]["bbox"][3] >= 180
                and masks[i]["bbox"][3] <= 220
                and masks[i]["bbox"][3] >= masks[i]["bbox"][2]
            )
            mask = np.array(masks[i]["segmentation"])

            # mask the color image
            color_mask = np.multiply(self.color_image, mask[:, :, np.newaxis])

            # mask the depth image
            depth_mask = (np.multiply(self.depth_image, mask), i)
            res1 = np.sum(mask)
            if res1 == 0:
                continue

            # print(res1, res2)
            # print(res1,res2)

            # sum the red channel of result
            res3 = np.sum(color_mask[:, :, 0]) / res1  # red channel
            res4 = np.sum(color_mask[:, :, 1]) / res1  # green channel
            res5 = np.sum(color_mask[:, :, 2]) / res1  # blue channel
            red = res3 > res4 + 10 and res3 > res5 + 10

            if selected and red:  # selected and red:                
                plt.figure(figsize=(20, 20))
                plt.imshow(color_mask) # todo uncomment
                plt.axis('off')

                plt.title(
                    "i: "
                    + str(i)
                    + "stab_score: "
                    + str(masks[i]["stability_score"])
                    + " area: "
                    + str(masks[i]["area"])
                    + "bbox"
                    + str(masks[i]["bbox"])
                    + "point_coords"
                    + str(masks[i]["point_coords"])
                    + "crop_box"
                    + str(masks[i]["crop_box"])
                    + "predicted_iou"
                    + str(masks[i]["predicted_iou"])
                    + "SELECTED: "
                    + str(selected)
                    + "is red "
                    + str(red)
                )

                # plt.show()

                # # save plot in a folder named plots with timestamp info added
                plt.savefig("plots/" + str(i) + str(datetime.now()) + ".png")


                masked_depth_images.append(depth_mask)

                # also plot the result
                # plt.figure(figsize=(20,20))
                # plt.imshow(color_mask)
                # plt.axis('off')
                # plt.show()

                # append result to rgbImages
                rgbImages.append(color_mask)
                predicted_pieces.append(0)

        # for each masked_depth_imge, images that are close to each other
        # and choose the one with largest area, and box shape
        new_masked_depth_images = []
        for i in range(len(masked_depth_images)):
            for j in range(len(masked_depth_images)):
                if i != j:
                    index1 = masked_depth_images[i][1]
                    index2 = masked_depth_images[j][1]
                    if (
                        np.linalg.norm(
                            np.stack(masks[index1]["bbox"][0:2], axis=0)
                            - np.stack(masks[index2]["bbox"][0:2], axis=0)
                        )
                        <= 50
                    ):
                        print(
                            f"index1:{index1} and index2:{index2} are close to each other"
                        )

                        # if masks are close enough (Eucledian distance), then compare their areas (they are masks of the same object)
                        if masks[index1]["area"] < masks[index2]["area"]:
                            masks[index1]["area"] = 0

                        else:
                            masks[index2]["area"] = 0
        # remove duplicates
        for i in range(len(masked_depth_images)):
            index = masked_depth_images[i][1]
            if masks[index]["area"] != 0:
                new_masked_depth_images.append(masked_depth_images[i][0])

        print("size of sent new masked depth images: ", len(new_masked_depth_images))

        if len(new_masked_depth_images) == 0:
            output.set_value(
                [[-1]
                    ,
                    [0],
                ]
            )
        else:
            output.set_value(
                [np.stack(new_masked_depth_images, axis=0)
                    ,
                    [0 for i in range(len(new_masked_depth_images))],
                ]
            )

# mask

In [37]:
!rm -rf ./plots/*
!rm -rf ./color_image/*

In [38]:
class CreatePointClouds(LeafSystem):
    """
    System to convert masked depth images into pointclouds.
    """

    # def __init__(self, rgbd_sensor, X_WC):
    def __init__(self, rgbd_sensor):
        LeafSystem.__init__(self)

        self.cam_info = rgbd_sensor.depth_camera_info()
        self.rgbd_sensor = rgbd_sensor

        # sensor_context = rgbd_sensor.GetMyMutableContextFromRoot(context)
        # self.X_WC = rgbd_sensor.body_pose_in_world_output_port().Eval(sensor_context)

        RigidTransform()

        self.DeclareAbstractInputPort(
            "depth_image_stack", AbstractValue.Make([np.ndarray])
        )

        self.DeclareAbstractInputPort(
            "rgbd_sensor_body_pose", AbstractValue.Make(RigidTransform())
        )

        self.DeclareAbstractOutputPort(
            "pcd_stack", lambda: AbstractValue.Make([np.ndarray]), self.calc_output
        )

    def get_intrinsics(self):
        # read camera intrinsics
        cx = self.cam_info.center_x()
        cy = self.cam_info.center_y()
        fx = self.cam_info.focal_x()
        fy = self.cam_info.focal_y()
        return cx, cy, fx, fy

    def project_depth_to_pC(self, depth_pixel):
        """
        project depth pixels to points in camera frame
        using pinhole camera model
        Input:
            depth_pixels: numpy array of (nx3) or (3,)
        Output:
            pC: 3D point in camera frame, numpy array of (nx3)
        """
        # switch u,v due to python convention
        v = depth_pixel[:, 0]
        u = depth_pixel[:, 1]
        Z = depth_pixel[:, 2]
        cx, cy, fx, fy = self.get_intrinsics()
        X = (u - cx) * Z / fx
        Y = (v - cy) * Z / fy
        pC = np.c_[X, Y, Z]
        return pC

    def get_pointcloud_np(self, depth_im, X_WC):
        u_range = np.arange(depth_im.shape[0])
        v_range = np.arange(depth_im.shape[1])
        depth_v, depth_u = np.meshgrid(v_range, u_range)
        depth_pnts = np.dstack([depth_u, depth_v, depth_im])
        depth_pnts = depth_pnts.reshape([depth_pnts.shape[0] * depth_pnts.shape[1], 3])
        pC = self.project_depth_to_pC(depth_pnts)
        p_C = pC[pC[:, 2] > 0]
        p_W = X_WC.multiply(p_C.T).T
        return p_W

    def get_drake_pcd(self, pcd_np, X_WC):
        N = pcd_np.shape[0]
        pcd = PointCloud(N, Fields(BaseField.kXYZs | BaseField.kRGBs))
        pcd.mutable_xyzs()[:] = pcd_np.T  # Want (3, N) while pcd_np is (N, 3)
        pcd.EstimateNormals(radius=0.1, num_closest=30)
        pcd.FlipNormalsTowardPoint(X_WC.translation())
        pcd = pcd.VoxelizedDownSample(voxel_size=0.002)
        return pcd

    def calc_output(self, context, output):
        # sensor_context = self.rgbd_sensor.GetMyMutableContextFromRoot(context)
        # X_WC = self.rgbd_sensor.body_pose_in_world_output_port().Eval(context)

        X_WC = self.GetInputPort("rgbd_sensor_body_pose").Eval(context)
        depth_image_stack, pieces = self.GetInputPort("depth_image_stack").Eval(context)
        # check if depth_image_stack first element is the array [-1]
        
        print("depth_image_stack: ", depth_image_stack, " len:", len(depth_image_stack) == 1)
        print("is istance: ", isinstance(depth_image_stack[0], int))
        if (isinstance(depth_image_stack[0], int)):
            print("in calc_output if statement depth_image_stack[0] == [-1]")
            output.set_value(
                [[-1]
                    ,
                    [0],
                ]
            )
            return

        pcds = []
        for i in range(depth_image_stack.shape[0]):
            depth_image = depth_image_stack[i]
            pcd_np = self.get_pointcloud_np(depth_image, X_WC)
            pcd = self.get_drake_pcd(pcd_np, X_WC)
            pcds.append(pcd)
        print("in calc_output")
        output.set_value([pcds, pieces])

In [39]:
a = np.stack([-1], axis=0)
isinstance(a, int)
type(a[0])

numpy.int64

# Make Frames

In [40]:
def VectorizedMakeGripperFrames(X_G, X_O, noOfObjects, initial_time=0):
    """
    Takes a partial specification with X_G[]["initial"] and X_O[]["initial"] and
    X_0[]["goal"], X_G[]["pick"] and returns a X_G[] and times[] with all of the pick and place
    frames populated.
    """

    # Now let's set the timing
    times = [dict() for x in range(noOfObjects)]
    # X_GgraspO = RigidTransform( RotationMatrix.MakeXRotation(np.pi)  ,[0.03,0.05,0.06])
    # X_OGgrasp = X_GgraspO.inverse()

    X_GgraspGpregrasp = RigidTransform([0, -0.2, 0])

    for i in range(noOfObjects):
        # X_G[i]["pick"] = X_O[i]["initial"] @ X_OGgrasp

        X_G[i]["prepick"] = X_G[i]["pick"] @ X_GgraspGpregrasp
        X_G[i]["pick_start"] = X_G[i]["pick"]
        X_G[i]["pick_end"] = X_G[i]["pick"]
        X_G[i]["postpick"] = X_G[i]["prepick"]

        X_G[i]["place"] = X_O[i]["goal"]
        X_G[i]["preplace"] = X_G[i]["place"] @ X_GgraspGpregrasp


        X_G[i]["clearance"] = RigidTransform(
            RotationMatrix.MakeXRotation(np.pi),
            np.array([0, 0, 0.5])
            + (X_G[i]["postpick"].translation()+ X_G[i]["preplace"].translation())/2,
        )

        # I'll interpolate a halfway orientation by converting to axis angle and halving the angle.
        X_GprepickGpreplace = X_G[i]["prepick"].inverse() @ X_G[i]["preplace"]
        angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()


        X_GprepickGclearance = RigidTransform(
            AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
            X_GprepickGpreplace.translation() / 2.0
            + np.array(X_G[i]["clearance"].translation()),
        )
        if i == 0:
            times[i] = {"initial": initial_time}
            X_G[0]["initial_to_prepick"] = X_G[0]["initial"]
        else:
            times[i] = {"initial": times[i - 1]["postplace_clearance"] + 4}
            X_G[i]["initial"] = X_G[i - 1]["postplace_clearance"]
            # X_G[i]["initial"] = X_G[0]["initial"]
            X_G[i]["initial_to_prepick"] = RigidTransform(
                RotationMatrix.MakeXRotation(np.pi), [0.0, 0.0, 0.40]
            )

        X_GinitialGprepick = X_G[i]["initial"].inverse() @ X_G[i]["prepick"]
        times[i]["initial_to_prepick"] = (
            times[i]["initial"]
            + (10.0 * np.linalg.norm(X_GinitialGprepick.translation())) / 2
        )

        times[i]["prepick"] = times[i]["initial"] + 10.0 * np.linalg.norm(
            X_GinitialGprepick.translation()
        )

        # Allow some time for the gripper to close.
        times[i]["pick_start"] = times[i]["prepick"] + 2.0
        times[i]["pick_end"] = times[i]["pick_start"] + 2.0
        times[i]["postpick"] = times[i]["pick_end"] + 2.0

        time_to_from_clearance = 5.0 * np.linalg.norm(
            X_GprepickGclearance.translation()
        )
        times[i]["clearance"] = times[i]["postpick"] + time_to_from_clearance
        times[i]["preplace"] = times[i]["clearance"] + time_to_from_clearance
        times[i]["place_start"] = times[i]["preplace"] + 2.0
        times[i]["place_end"] = times[i]["place_start"] + 2.0
        X_G[i]["place_start"] = X_G[i]["place"]
        X_G[i]["place_end"] = X_G[i]["place"]
        times[i]["postplace"] = times[i]["place_end"] + 2.0
        X_G[i]["postplace"] = X_G[i]["preplace"]

        times[i]["postplace_clearance"] = times[i]["postplace"] + 2.0
        X_G[i]["postplace_clearance"] = X_G[i]["clearance"]  # X_G[i]["preplace"]

        # print("GRAVGRAVGRAVGRAV")
        # print(times)
        # print("times bitti", "X_G now")
        # print(X_G)

    return X_G, times

In [41]:
# # write a function that returns the corner points of a given image , where 1's are objects and 0's are background
# def get_corners(img):
#     # img = cv2.imread(img)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = np.float32(gray)
#     corners = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)
#     corners = np.int0(corners)
#     corner_list = []
#     for corner in corners:
#         x, y = corner.ravel()
#         corner_list.append([x,y])
#     return corner_list

# gripper frame

In [42]:
def VectorizedMakeGripperPoseTrajectory(X_G, times, noOfObjects):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """

    sample_times = []
    sample_times_toprint = []
    poses = []
    timetoadd = 0
    for i in range(noOfObjects):
        for name in [
            # "initial", "initial_to_prepick", "prepick", "pick_start", "pick_end", "postpick",
            "initial", "prepick", "pick_start", "pick_end", "postpick",
                    "clearance", "preplace", "place_start", "place_end",
                    # "preplace", "place_start", "place_end",
                    "postplace", "postplace_clearance"]:
            sample_times_toprint.append((times[i][name], i, name, X_G[i][name].translation()))
            sample_times.append(times[i][name])
            poses.append(X_G[i][name])
        
            # poses.append(X_G[0]["initial"])
            
        #if i != 1:
        sample_times.append(times[i]["postplace"] + 3)
        sample_times_toprint.append((times[i]["postplace"] + 3, i, "Outer Loop clearance", X_G[i]["clearance"].translation()))
        poses.append(X_G[i]["clearance"])
    print("GRAVGRAVGRAVGRAV")
    print(sample_times_toprint)
    print("times bitti", "X_G now")
    #print(poses)
    
    return PiecewisePose.MakeLinear(sample_times, poses)

In [43]:
def MakeStorageTrajectory(X_G, times):
    sample_times = []
    poses = []
    for name in ["initial", "source_start", "source_end","clearance", "destination_start","destination_end","finish" ]:
        sample_times.append(times[name])
        poses.append(X_G[name])

    return PiecewisePose.MakeLinear(sample_times, poses)


saat()

2023-05-31 16:14:20.323484 


In [44]:
def VectorizedMakeGripperCommandTrajectory(times, noOfObjects):
    opened = np.array([0.107])
    closed = np.array([0.0])

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [times[0]["initial"], times[0]["pick_start"]], np.hstack([[opened], [opened]])
    )

    traj_wsg_command.AppendFirstOrderSegment(times[0]["pick_end"], closed)
    traj_wsg_command.AppendFirstOrderSegment(times[0]["place_start"], closed)
    traj_wsg_command.AppendFirstOrderSegment(times[0]["place_end"], opened)
    traj_wsg_command.AppendFirstOrderSegment(times[0]["postplace"], opened)

    for i in range(1, noOfObjects):
        traj_wsg_command.AppendFirstOrderSegment(times[i]["pick_start"], opened)
        traj_wsg_command.AppendFirstOrderSegment(times[i]["pick_end"], closed)
        traj_wsg_command.AppendFirstOrderSegment(times[i]["place_start"], closed)
        traj_wsg_command.AppendFirstOrderSegment(times[i]["place_end"], opened)
        # traj_wsg_command.AppendFirstOrderSegment(times[i]["postplace"], opened)

    return traj_wsg_command

In [45]:

# import matplotlib.pyplot as plt
# # plt.imshow(np.squeeze(icp.inputs[5].data))
# # plt.title('Depth image')

# # res = icp.inputs[5].data

# masks = mask_generator.generate(res[:,:,:3])
# for i in range(len(masks)):
#     if (masks[i]['stability_score'] > 0.99 and masks[i]['area'] > 30000 ):
#         image = np.array(masks[i]["segmentation"])
#         plt.figure(figsize=(20,20))
#         plt.imshow(image)
#         plt.axis('off')
#         plt.show()

# Robot Planner

In [46]:
X_G = 0
X_G2 = 0


class PickAndPlaceTrajectory3(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._gripper_body_index = plant.GetBodyByName(
            "body", plant.GetModelInstanceByName("wsg")
        ).index()

        self._gripper_body_index2 = plant.GetBodyByName(
            "body", plant.GetModelInstanceByName("wsg2")
        ).index()
        # self._gripper_body_index2 = plant.GetBodyByName("body", plant.GetModelInstanceByName("wsg2")).index()

        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )

        self._iiwa_link_0_original_body_index = plant.GetBodyByName(
            "iiwa_link_0_original", plant.GetModelInstanceByName("iiwa")
        ).index()
        self._iiwa_link_0_original_body_index2 = plant.GetBodyByName(
            "iiwa_link_0_z", plant.GetModelInstanceByName("storage")
        ).index()

        self.DeclareAbstractInputPort("X_WO", AbstractValue.Make([RigidTransform()]))

        self.DeclareInitializationUnrestrictedUpdateEvent(self.Plan)
        self._traj_X_G_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose())
        )
        self._traj_wsg_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )

        self._traj_X_G_index2 = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose())
        )
        self._traj_wsg_index2 = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )

        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()), self.CalcGripperPose
        )

        self.DeclareAbstractOutputPort(
            "X_WG2", lambda: AbstractValue.Make(RigidTransform()), self.CalcGripperPose2
        )

        self.noOfObjects = 0

        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)
        self.DeclareVectorOutputPort("wsg_position2", 1, self.CalcWsgPosition2)

    def Plan(self, context, state):
        # X_G = {
        #     "initial":
        #         self.get_input_port(0).Eval(context)
        #         [int(self._gripper_body_index)]
        # }

        # X_O = {
        #     "initial": self.get_input_port(1).Eval(context),
        #     #"goal": RigidTransform( [1.15, 0.0, 0.45])
        #     "goal": RigidTransform(RotationMatrix.MakeXRotation(np.pi), [1.0, 0.0, 0.450])
        # }
        # print("body poses", self.get_input_port(0).Eval(context))
        self.noOfObjects = len(self.get_input_port(1).Eval(context))
        print("no of objects", self.noOfObjects)

        global X_G
        global X_G2

        X_O = [dict() for x in range(self.noOfObjects)]
        X_O2 = [dict() for x in range(self.noOfObjects)]

        X_G = [dict() for x in range(self.noOfObjects)]
        X_G2 = [dict() for x in range(self.noOfObjects)]

        for i in range(self.noOfObjects):
            X_O[i]["initial"] = self.get_input_port(1).Eval(context)[i]

            # X_O[i]["initial"]= RigidTransform(RotationMatrix.MakeXRotation(np.pi), [1.0, 0.0, 0.500])
            # X_O[i]["goal"] = RigidTransform(
            #     RotationMatrix.MakeZRotation(np.pi/9) @ RotationMatrix.MakeXRotation(np.pi), [1.0-(i*0.15), 0.0, 0.140 + (i)*0.245] # todo change goals gar'p
            # ) @ RigidTransform(   [0,0.05,0]   )

            # FOR STORAGE
            X_O[i]["goal"] = RigidTransform(
                RotationMatrix.MakeZRotation(-np.pi / 2)
                @ RotationMatrix.MakeXRotation(-np.pi / 2),
                [-1.3, -0.0 - (2 * i - 3) * 0.1, 0.15],  # todo change goals gar'p
            ) @ RigidTransform([0, 0.05, 0])

            # FOR second iiwa
            # TODO fix
            X_O2[i]["initial"] = RigidTransform(
                RotationMatrix.MakeZRotation(np.pi/2)
                @ RotationMatrix.MakeXRotation(np.pi/2)
                @ RotationMatrix.MakeYRotation(np.pi),
                [
                    2.5-0.30,
                    4.5 - (2 * i - 3) * 0.1,
                    -0.20,
                ],  # todo change goals gar'p
            ) @ RigidTransform([0, 0.05, 0])

            # Xgoal
            X_O2[i]["goal"] = RigidTransform(
                RotationMatrix.MakeZRotation(0)
                @ RotationMatrix.MakeXRotation(np.pi),
                [
                    3 + 1.2 - (i * 0.15) - 0.10,
                    3 + 0.0,
                    0.22 + (i + 1) * 0.245,
                ],  # todo change goals gar'p
            ) @ RigidTransform([0, -0.15, 0])

        X_G[0]["initial"] = self.get_input_port(0).Eval(context)[
            int(self._gripper_body_index)
        ]

        # !!!!!!!!!
        # X_G2[0]["initial"] = self.get_input_port(0).Eval(context)[int(self._gripper_body_index2)]
        X_G2[0]["initial"] = self.get_input_port(0).Eval(context)[
            int(self._gripper_body_index2)
        ]

        # print("X_G[0][initial]", X_G[0]["initial"], "X_G2[0][initial]", X_G2[0]["initial"])
        print(
            "X_O[0][initial]", X_O[0]["initial"], "X_O2[0][initial]", X_O2[0]["initial"]
        )
        # ================== 1st trajectory ==================

        # trans= self.get_input_port(0).Eval(context)[int(self._iiwa_link_0_original_body_index)]
        # tr = trans.translation()

        # tr = tr+[-0.5, 0, 0]
        # trans.set_translation(tr)

        # trans = trans @ RigidTransform(RotationMatrix.MakeXRotation(np.pi / 2.0), [0, 0, 0])

        # trans.set_rotation(trans.rotation() @ RotationMatrix.MakeYRotation(np.pi))

        # X_O[1]["goal"] = trans # self.get_input_port(0).Eval(context)[int(self._iiwa_link_0_original_body_index)]

        #  rotation: !Rpy { deg: [0.0, 70.0, 180.0 ]}
        #  translation: [1.2, 0.05, 0.45]

        for i in range(self.noOfObjects):
            X_GgraspO = RigidTransform(
                RotationMatrix.MakeXRotation(np.pi), [0, 0.22, 0]
            )
            X_OGgrasp = X_GgraspO.inverse()
            # X_GgraspO2 = RigidTransform( RotationMatrix.MakeYRotation(np.pi)  ,[-0.03,-0.03,-0.06])
            # X_OGgrasp2 = X_GgraspO.inverse()

            X_G[i]["pick"] = X_O[i]["initial"] @ X_OGgrasp
            X_G2[i]["pick"] = X_O2[i]["initial"] @ X_OGgrasp #RigidTransform([5, 4.5, 0.0])
            # X_G[i]["pick"].set_translation(X_G[i]["pick"].translation()+[0, 0, 0.47])

            # hardcoded
            # X_G[i]["pick"] = RigidTransform(RotationMatrix.MakeXRotation(np.pi)  ,[-0.2,-1.0,0.5])

        # tmp = X_O["goal"] @ X_OGgrasp
        # I'll interpolate a halfway orientation by converting to axis angle and halving the angle.

        # X_GprepickGpreplace = X_G["pick"].inverse() @ tmp
        # angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
        # X_GprepickGclearance = RigidTransform(
        #     AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        #     X_GprepickGpreplace.translation() / 2.0 + np.array([0.8, -1, 2]),
        # )

        # # the arm should rotate to avoid collision, the Z rotation is for that!
        # X_GprepickGclearance.set_rotation(X_GprepickGclearance.rotation() @ RotationMatrix.MakeZRotation(-np.pi / 2.0))
        # tmpReal = X_G["pick"] @ X_GprepickGclearance

        # # hardcoded
        # tmpReal = RigidTransform( RotationMatrix.MakeXRotation(np.pi), [0.0, 0.0, 0.40])

        # X_G["place"] = X_O["goal"] @ X_OGgrasp

        # hardcode

        for i in range(self.noOfObjects):
            X_G[i]["place"] = X_O[i]["goal"]
            X_G2[i]["place"] = X_O2[i]["goal"]

        # degisicek
        times = VectorizedMakeGripperFrames(X_G, X_O, self.noOfObjects)[1]
        times2 = VectorizedMakeGripperFrames(X_G2, X_O2, self.noOfObjects, 20+times[-1]["postplace_clearance"])[1]

        for i in range(self.noOfObjects):
            if True:  # True:  # Useful for debugging
                X_G_dummy, times_dummy = X_G2, times2
                AddMeshcatTriad(meshcat, "X_Oinitial " + str(i), X_PT=X_O[i]["initial"])
                AddMeshcatTriad(meshcat, "X_goal " + str(i), X_PT=X_O2[i]["goal"])

                AddMeshcatTriad(
                    meshcat,
                    f"gripper2 " + str(i) + " " + str(times_dummy[i]["initial"]),
                    X_PT=X_G_dummy[i]["initial"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gprepick2 " + str(i) + " " + str(times_dummy[i]["prepick"]),
                    X_PT=X_G_dummy[i]["prepick"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gpick_start2 " + str(i) + " " + str(times_dummy[i]["pick_start"]),
                    X_PT=X_G_dummy[i]["pick_start"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gpick_end2 " + str(i) + " " + str(times_dummy[i]["pick_end"]),
                    X_PT=X_G_dummy[i]["pick_end"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gpostpick2 " + str(i) + " " + str(times_dummy[i]["postpick"]),
                    X_PT=X_G_dummy[i]["postpick"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gclearance2 " + str(i) + " " + str(times_dummy[i]["clearance"]),
                    X_PT=X_G_dummy[i]["clearance"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gpreplace2 " + str(i) + " " + str(times_dummy[i]["preplace"]),
                    X_PT=X_G_dummy[i]["preplace"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gplace_start2 " + str(i) + " " + str(times_dummy[i]["place_start"]),
                    X_PT=X_G_dummy[i]["place_start"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gplace_end2 " + str(i) + " " + str(times_dummy[i]["place_end"]),
                    X_PT=X_G_dummy[i]["place_end"],
                )
                AddMeshcatTriad(
                    meshcat,
                    "X_Gpostplace2 " + str(i) + " " + str(times_dummy[i]["postplace"]),
                    X_PT=X_G_dummy[i]["postplace"],
                )

                # AddMeshcatTriad(meshcat, f"tmpReal "+ str(i), X_PT=tmpReal)

        traj_X_G = VectorizedMakeGripperPoseTrajectory(X_G, times, self.noOfObjects)
        traj_X_G2 = VectorizedMakeGripperPoseTrajectory(X_G2, times2, self.noOfObjects)

        traj_wsg_command = VectorizedMakeGripperCommandTrajectory(
            times, self.noOfObjects
        )
        traj_wsg_command2 = VectorizedMakeGripperCommandTrajectory(
            times2, self.noOfObjects
        )

        state.get_mutable_abstract_state(int(self._traj_X_G_index)).set_value(traj_X_G)
        state.get_mutable_abstract_state(int(self._traj_wsg_index)).set_value(
            traj_wsg_command
        )

        state.get_mutable_abstract_state(int(self._traj_X_G_index2)).set_value(
            traj_X_G2
        )

        state.get_mutable_abstract_state(int(self._traj_wsg_index2)).set_value(
            traj_wsg_command2
        )

    def start_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .start_time()
        )

    def end_time(self, context):
        r =             context.get_abstract_state(int(self._traj_X_G_index)).get_value().end_time()
        if           r<=  context.get_abstract_state(int(self._traj_X_G_index2)).get_value().end_time():
            r = context.get_abstract_state(int(self._traj_X_G_index2)).get_value().end_time()

        return r

    def CalcGripperPose(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.set_value(
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .GetPose(context.get_time())
        )

    def CalcWsgPosition(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.SetFromVector(
            context.get_abstract_state(int(self._traj_wsg_index))
            .get_value()
            .value(context.get_time())
        )

    def start_time2(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index2))
            .get_value()
            .start_time()
        )

    def end_time2(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index2))
            .get_value()
            .end_time()
        )

    def CalcGripperPose2(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.set_value(
            context.get_abstract_state(int(self._traj_X_G_index2))
            .get_value()
            .GetPose(context.get_time())
        )

    def CalcWsgPosition2(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.SetFromVector(
            context.get_abstract_state(int(self._traj_wsg_index2))
            .get_value()
            .value(context.get_time())
        )

# Storage Planner

In [47]:
class StoragePlanner(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        # self._gripper_body_index = plant.GetBodyByName("body", plant.GetModelInstanceByName("wsg")).index()
        # self._gripper_body_index2 = plant.GetBodyByName("body", plant.GetModelInstanceByName("wsg2")).index()

        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )  # body pose 2 plannera input olarak geliyor

        self._iiwa_link_0_original_body_index2 = plant.GetBodyByName(
            "iiwa_link_0_z", plant.GetModelInstanceByName("storage")
        ).index()

        self.DeclareInitializationUnrestrictedUpdateEvent(self.Plan)

        self._traj_X_WS_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose())
        )
        self._traj_wsg_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )

        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()), self.CalcGripperPose
        )

        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

    def Plan(self, context, state):
        # set some points for storage to move
        X_WS = dict()
        X_WS["initial"] = self.get_input_port(0).Eval(context)[
            int(self._iiwa_link_0_original_body_index2)
        ]

        #     for name in ["initial", "source_start", "source_end","clearance", "destination_start","destination_end","finish" ]:

        X_WS["source_start"] = RigidTransform([-1, 0, 0])
        X_WS["source_end"] = X_WS["source_start"]
        X_WS["clearance"] = RigidTransform([-1.2, 3, 0.0])
        X_WS["destination_start"] = RigidTransform([2.5, 4.5, 0.0])
        X_WS["destination_end"] = X_WS["destination_start"]
        X_WS["finish"] = X_WS["destination_end"]

        times = dict()
        times["initial"] = 0.0
        times["source_start"] = 5.0
        times["source_end"] = times["source_start"] + 40.0
        times["clearance"] =  times["source_end"] + 20.0
        times["destination_start"] = times["clearance"] + 20.0
        times["destination_end"] = times["destination_start"] +400.0
        times["finish"] = times["destination_end"] + 10.0
        
        if  True:  # Useful for debugging
            AddMeshcatTriad(meshcat, "X_WS/[initial]", X_PT=X_WS["initial"])
            AddMeshcatTriad(meshcat, "X_WS/[source_start]", X_PT=X_WS["source_start"])
            AddMeshcatTriad(meshcat, "X_WS/[source_end]", X_PT=X_WS["source_end"])
            AddMeshcatTriad(meshcat, "X_WS/[clearance]", X_PT=X_WS["clearance"])
            AddMeshcatTriad(meshcat, "X_WS/[destination_start]", X_PT=X_WS["destination_start"])
            AddMeshcatTriad(meshcat, "X_WS/[destination_end]", X_PT=X_WS["destination_end"])
            AddMeshcatTriad(meshcat, "X_WS/[finish]", X_PT=X_WS["finish"])
            # AddMeshcatTriad(meshcat, "X_WS[p0]", X_PT=X_WS["p0"])
            # AddMeshcatTriad(meshcat, f"X_WS[p1] ", X_PT=X_WS["p1"])

        traj_X_WS = MakeStorageTrajectory(X_WS, times)

        opened = np.array([0.107])
        closed = np.array([0.0])

        traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
            [times["initial"], times["source_start"]], np.hstack([[opened], [opened]])
        )

        traj_wsg_command.AppendFirstOrderSegment(times["source_end"], closed)

        state.get_mutable_abstract_state(int(self._traj_X_WS_index)).set_value(
            traj_X_WS
        )
        state.get_mutable_abstract_state(int(self._traj_wsg_index)).set_value(
            traj_wsg_command
        )

    def CalcWsgPosition(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.SetFromVector(
            context.get_abstract_state(int(self._traj_wsg_index))
            .get_value()
            .value(context.get_time())
        )

    def start_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_WS_index))
            .get_value()
            .start_time()
        )

    def end_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_WS_index))
            .get_value()
            .end_time()
        )

    def CalcGripperPose(self, context, output):
        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        output.set_value(
            context.get_abstract_state(int(self._traj_X_WS_index))
            .get_value()
            .GetPose(context.get_time())
        )

# setup

In [48]:
def setup(plant):
    # add boxes
    # book = AddShape(plant, Box(0.205, 0.06, 0.154), "book", color=[1, 0, 0, 1.0])
    # plant.SetDefaultFreeBodyPose(
    #     plant.GetBodyByName("book"),
    #     RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [-0.25, -0.95, 0.45]),
    # )

    # book2 = AddShape(plant, Box(0.205, 0.06, 0.154), "book2", color=[0, 1, 0, 1.0])

    # plant.SetDefaultFreeBodyPose(
    #     plant.GetBodyByName("book2"),
    #     RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [-0.10, -0.95, 0.45]),
    # )

    # book3 = AddShape(plant, Box(0.205, 0.06, 0.154), "book3", color=[0, 1, 0, 1.0])
    # plant.SetDefaultFreeBodyPose(
    #     plant.GetBodyByName("book3"),
    #     RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [0.05, -0.95, 0.45]),
    # )

    xdiff = 4.0
    ydiff = 4.0
    zdiff = 0.05
    showAllBooks = False

    if showAllBooks:
        for k in range(2):
            for i in range(3):
                book = AddShape(plant, Box(0.205, 0.06, 0.154), "def_book" + str(i) + "iiwa" + str(k+1), color=[0 + k, 1 - k, 0, 1.0])
                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("def_book" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [-0.25 + 0.15 * i+ k * xdiff, -0.95+k * ydiff, 0.78+ zdiff * k]),
                )
                book = AddShape(plant, Box(0.205, 0.06, 0.154), "2_def_book" + str(i)+ "iiwa" + str(k+1), color=[0 + k, 1 - k, 0, 1.0])
                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("2_def_book" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [0.35 + 0.15 * i+ k * xdiff, -0.95+k * ydiff, 0.78+ zdiff* k]),
                )
            for i in range(3):
                if i==0 and k == 0:
                    book = AddShape(plant, Box(0.205, 0.06, 0.154), "2_def_book_1" + str(i)+ "iiwa" + str(k+1), color=[1, 0, 0, 1.0])
                else:
                    book = AddShape(plant, Box(0.205, 0.06, 0.154), "2_def_book_1" + str(i)+ "iiwa" + str(k+1), color=[0+k, 1-k, 0, 1.0])
                
                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("2_def_book_1" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [0.35 + 0.15 * i+k * xdiff, -0.95+k * ydiff, 0.45+ zdiff* k]),
                ) 
                if (i==2 or i==1) and k == 1:
                    continue

                if False:
                    book = AddShape(plant, Box(0.205, 0.06, 0.154), "def_book_1" + str(i)+ "iiwa" + str(k+1), color=[1, 0, 0, 1.0])
                else:
                    book = AddShape(plant, Box(0.205, 0.06, 0.154), "def_book_1" + str(i)+ "iiwa" + str(k+1), color=[0+k, 1-k, 0, 1.0])

                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("def_book_1" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [-0.25 + k * xdiff + 0.15 * i, -0.95 + k * ydiff, 0.45+ zdiff* k]),
                )

            for i in range(3):
                if i==0 and k == 1 or i==1 and k == 1:
                    continue

                book = AddShape(plant, Box(0.205, 0.06, 0.154), "def_book_2" + str(i)+ "iiwa" + str(k+1), color=[0 + k, 1 - k, 0, 1.0])
                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("def_book_2" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [-0.25 + 0.15 * i+k * xdiff, -0.95+k * ydiff, 0.18+ zdiff* k]),
                )
                book = AddShape(plant, Box(0.205, 0.06, 0.154), "2_def_book_2" + str(i)+ "iiwa" + str(k+1), color=[0 + k, 1 - k, 0, 1.0])
                
                plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("2_def_book_2" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [0.35 + 0.15 * i+k * xdiff, -0.95+k * ydiff, 0.18+ zdiff* k]),
            )
    else:
        i = 0
        k = 0
        book = AddShape(plant, Box(0.205, 0.06, 0.154), "2_def_book_1" + str(i)+ "iiwa" + str(k+1), color=[1, 0, 0, 1.0])
        plant.SetDefaultFreeBodyPose(
                    plant.GetBodyByName("2_def_book_1" + str(i)+ "iiwa" + str(k+1)),
                    RigidTransform(RollPitchYaw(0, np.pi / 2, np.pi / 2), [0.35 + 0.15 * i+k * xdiff, -0.95+k * ydiff, 0.45+ zdiff* k]),
                ) 


    # add the ground!
    ground = AddShape(
        plant, Box(20.0, 20.0, 0.1), "ground", color=[0.7539, 0.7539, 0.7539, 1.0]
    )
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("ground", ground),
        RigidTransform([0, 0, -0.1]),
    )

    # place the books!
    # plant.SetDefaultFreeBodyPose(plant.GetBodyByName("base_link_cracker"), X_O['initial'])
    # plant.SetDefaultFreeBodyPose(plant.GetBodyByName("base_link_sugar"), X_O2['initial'])

    ###################################################
    # The book storage of iiwa should be added here
    ###################################################
    # wall01 = AddShape(plant,
    #                     Box(1., 1., .1),
    #                     "wall01",
    #                     color=[0, 1, 0, 1.0])
    # plant.SetDefaultFreeBodyPose(plant.GetBodyByName("wall01"),RigidTransform([4, 4, 6]))

    width = 0.33
    depth = 0.09
    height = 0.07
    thick = 0.004
    thickness = thick
    m = 1  # should be an positive odd number
    n = 5  # should be an positive integer
    storageRT = RigidTransform([0.5, 0, 0])
    storages = []
    for i in range(0, m):
        for j in range(0, n):
            storages.append(
                AddShape(
                    plant,
                    Box(width, depth, thick),
                    "storage_bottom" + "_" + str(i) + "_" + str(j),
                    color=[0.5, 0, 0.5, 1.0],
                )
            )
            list_index = (i * n + j) * 5
            first_dimension_difference = width + 2 * thickness
            first_dimension_index_difference = i - 0

            second_dimension_difference = depth + 2 * thickness
            second_dimension_index_difference = j - int(n / 2)

            # plant.WeldFrames(plant.GetFrameByName("iiwa_link_0"),


            plant.WeldFrames(
                plant.GetFrameByName(
                    "iiwa_link_0_original", plant.GetModelInstanceByName("storage")
                ),
                plant.GetFrameByName(
                    "storage_bottom" + "_" + str(i) + "_" + str(j), storages[list_index]
                ),
                RigidTransform(
                    [
                        -(width / 2 + 0.15)
                        - (width + 2 * thickness) * first_dimension_index_difference,
                        -(depth + 2 * thickness) * second_dimension_index_difference,
                        0,
                    ]
                ),
            )

            # storage_joint = plant.AddJoint(
            #     RevoluteJoint(
            #         "storage_theta",
            #         plant.GetFrameByName(
            #             "storage_bottom" + "_" + str(i) + "_" + str(j)
            #         ),
            #         plant.GetFrameByName(
            #             "iiwa_link_0_z", plant.GetModelInstanceByName("storage")
            #         ),
            #         [0, 0, 1],
            #     )
            # )

            # plant.AddJointActuator("freebox_body_theta_actuator", freebox_body_theta)

            storages.append(
                AddShape(
                    plant,
                    Box(height, depth, thick),
                    "storage_side01" + "_" + str(i) + str("_") + str(j),
                    color=[1, 1, 0, 1.0],
                )
            )
            # n1 = "storage_bottom" +"_" + str(i) +"_" + str(j)
            n1 = "storage_bottom" + "_" + str(i) + "_" + str(j)
            n2 = "storage_side01" + "_" + str(i) + str("_") + str(j)
            plant.WeldFrames(
                plant.GetFrameByName(n1, storages[list_index]),
                plant.GetFrameByName(n2, storages[list_index + 1]),
                RigidTransform(
                    RotationMatrix.MakeYRotation(np.pi / 2.0),
                    [-(width / 2 + thick / 2), 0, height / 2 + thick / 2],
                ),
            )

            storages.append(
                AddShape(
                    plant,
                    Box(width, height, thick),
                    "storage_side02" + "_" + str(i) + str("_") + str(j),
                    color=[0, 1, 1, 1.0],
                )
            )
            plant.WeldFrames(
                plant.GetFrameByName(
                    "storage_bottom" + "_" + str(i) + "_" + str(j), storages[list_index]
                ),
                plant.GetFrameByName(
                    "storage_side02" + "_" + str(i) + str("_") + str(j),
                    storages[list_index + 2],
                ),
                RigidTransform(
                    RotationMatrix.MakeXRotation(np.pi / 2.0),
                    [0, depth / 2 + thick / 2, height / 2 + thick / 2],
                ),
            )
            storages.append(
                AddShape(
                    plant,
                    Box(width, height, thick),
                    "storage_side03" + "_" + str(i) + str("_") + str(j),
                    color=[0, 1, 1, 0.5],
                )
            )
            plant.WeldFrames(
                plant.GetFrameByName(
                    "storage_bottom" + "_" + str(i) + "_" + str(j), storages[list_index]
                ),
                plant.GetFrameByName(
                    "storage_side03" + "_" + str(i) + str("_") + str(j),
                    storages[list_index + 3],
                ),
                RigidTransform(
                    RotationMatrix.MakeXRotation(np.pi / 2.0),
                    [0, -depth / 2 - thick / 2, height / 2 + thick / 2],
                ),
            )
            storages.append(
                AddShape(
                    plant,
                    Box(height, depth, thick),
                    "storage_side04" + "_" + str(i) + str("_") + str(j),
                    color=[1, 0, 0, 1.0],
                )
            )
            plant.WeldFrames(
                plant.GetFrameByName(
                    "storage_bottom" + "_" + str(i) + str("_") + str(j),
                    storages[list_index],
                ),
                plant.GetFrameByName(
                    "storage_side04" + "_" + str(i) + str("_") + str(j),
                    storages[list_index + 4],
                ),
                RigidTransform(
                    RotationMatrix.MakeYRotation(np.pi / 2.0),
                    [width / 2 + thick / 2, 0, height / 2 + thick / 2],
                ),
            )


saat("setup function is defined")

2023-05-31 16:14:20.843724 setup function is defined


In [49]:
def CustomAddWsg(
    plant, iiwa_model_instance, roll=np.pi / 2.0, welded=False, sphere=False
):
    parser = Parser(plant)
    if welded:
        if sphere:
            gripper = parser.AddModelFromFile(
                FindResource("models/schunk_wsg_50_welded_fingers_sphere.sdf"),
                "gripper",
            )
        else:
            gripper = parser.AddModelFromFile(
                FindResource("models/schunk_wsg_50_welded_fingers.sdf"), "gripper"
            )
    else:
        gripper = parser.AddModelFromFile(
            FindResourceOrThrow(
                "drake/manipulation/models/"
                "wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"
            )
        )

    X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, roll), [0, 0, 0.09])
    plant.WeldFrames(
        plant.GetFrameByName("iiwa_link_0_original", iiwa_model_instance),
        plant.GetFrameByName("body", gripper),
        X_7G,
    )
    return gripper

In [50]:
import os
import sys
import warnings

import numpy as np
from pydrake.all import (
    AbstractValue,
    Adder,
    AddMultibodyPlantSceneGraph,
    BallRpyJoint,
    BaseField,
    Box,
    CameraInfo,
    Capsule,
    ClippingRange,
    CoulombFriction,
    Cylinder,
    Demultiplexer,
    DepthImageToPointCloud,
    DepthRange,
    DepthRenderCamera,
    DiagramBuilder,
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters,
    GeometryInstance,
    InverseDynamicsController,
    LeafSystem,
    MakeMultibodyStateToWsgStateSystem,
    MakePhongIllustrationProperties,
    MakeRenderEngineVtk,
    ModelInstanceIndex,
    MultibodyPlant,
    Parser,
    PassThrough,
    PrismaticJoint,
    RenderCameraCore,
    RenderEngineVtkParams,
    RevoluteJoint,
    Rgba,
    RgbdSensor,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SchunkWsgPositionController,
    SpatialInertia,
    Sphere,
    StateInterpolatorWithDiscreteDerivative,
    UnitInertia,
)

In [51]:
def CustomAddRgbdSensors(
    builder,
    plant,
    scene_graph,
    also_add_point_clouds=True,
    model_instance_prefix="camera",
    depth_camera=None,
    renderer=None,
):
    """
    Adds a RgbdSensor to the first body in the plant for every model instance
    with a name starting with model_instance_prefix.  If depth_camera is None,
    then a default camera info will be used.  If renderer is None, then we will
    assume the name 'my_renderer', and create a VTK renderer if a renderer of
    that name doesn't exist.
    """
    if sys.platform == "linux" and os.getenv("DISPLAY") is None:
        from pyvirtualdisplay import Display

        virtual_display = Display(visible=0, size=(1400, 900))
        virtual_display.start()

    if not renderer:
        renderer = "my_renderer"

    if not scene_graph.HasRenderer(renderer):
        scene_graph.AddRenderer(renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))

    if not depth_camera:
        depth_camera = DepthRenderCamera(
            RenderCameraCore(
                renderer,
                CameraInfo(width=640, height=480, fov_y=np.pi / 4.0),
                ClippingRange(near=0.1, far=10.0),
                RigidTransform(),
            ),
            DepthRange(0.1, 10.0),
        )

    for index in range(plant.num_model_instances()):
        model_instance_index = ModelInstanceIndex(index)
        model_name = plant.GetModelInstanceName(model_instance_index)

        if model_name.startswith(model_instance_prefix):
            body_index = plant.GetBodyIndices(model_instance_index)[0]
            rgbd = builder.AddSystem(
                RgbdSensor(
                    parent_id=plant.GetBodyFrameIdOrThrow(body_index),
                    X_PB=RigidTransform(),
                    depth_camera=depth_camera,
                    show_window=False,
                )
            )
            rgbd.set_name(model_name)
            builder.Connect(
                scene_graph.get_query_output_port(),
                rgbd.query_object_input_port(),
            )

            # Export the camera outputs
            builder.ExportOutput(
                rgbd.color_image_output_port(), f"{model_name}_rgb_image"
            )
            builder.ExportOutput(
                rgbd.depth_image_32F_output_port(), f"{model_name}_depth_image"
            )
            builder.ExportOutput(
                rgbd.label_image_output_port(), f"{model_name}_label_image"
            )

            if also_add_point_clouds:
                # Add a system to convert the camera output into a point cloud
                to_point_cloud = builder.AddSystem(
                    DepthImageToPointCloud(
                        camera_info=rgbd.depth_camera_info(),
                        fields=BaseField.kXYZs | BaseField.kRGBs,
                    )
                )
                builder.Connect(
                    rgbd.depth_image_32F_output_port(),
                    to_point_cloud.depth_image_input_port(),
                )
                builder.Connect(
                    rgbd.color_image_output_port(),
                    to_point_cloud.color_image_input_port(),
                )

                class ExtractBodyPose(LeafSystem):
                    def __init__(self, body_index):
                        LeafSystem.__init__(self)
                        self.body_index = body_index
                        self.DeclareAbstractInputPort(
                            "poses",
                            plant.get_body_poses_output_port().Allocate(),
                        )
                        self.DeclareAbstractOutputPort(
                            "pose",
                            lambda: AbstractValue.Make(RigidTransform()),
                            self.CalcOutput,
                        )

                    def CalcOutput(self, context, output):
                        poses = self.EvalAbstractInput(context, 0).get_value()
                        pose = poses[int(self.body_index)]
                        output.get_mutable_value().set(
                            pose.rotation(), pose.translation()
                        )

                camera_pose = builder.AddSystem(ExtractBodyPose(body_index))
                builder.Connect(
                    plant.get_body_poses_output_port(),
                    camera_pose.get_input_port(),
                )
                builder.Connect(
                    camera_pose.get_output_port(),
                    to_point_cloud.GetInputPort("camera_pose"),
                )

                # Export the point cloud output.
                builder.ExportOutput(
                    to_point_cloud.point_cloud_output_port(),
                    f"{model_name}_point_cloud",
                )

                # ----------------------------------------------------------------
                mask_system = builder.AddSystem(MaskSystem(rgbd))

                builder.Connect(
                    rgbd.color_image_output_port(), mask_system.get_input_port(0)
                )
                builder.Connect(
                    rgbd.depth_image_32F_output_port(),
                    mask_system.GetInputPort("depth_image"),
                )
                builder.ExportOutput(
                    mask_system.GetOutputPort("masked_depth_image"),
                    f"{model_name}_masked_depth_image",
                )

                pcds = builder.AddSystem(CreatePointClouds(rgbd))
                builder.Connect(
                    mask_system.GetOutputPort("masked_depth_image"),
                    pcds.GetInputPort("depth_image_stack"),
                )
                builder.Connect(
                    rgbd.body_pose_in_world_output_port(),
                    pcds.GetInputPort("rgbd_sensor_body_pose"),
                )
                builder.ExportOutput(
                    pcds.GetOutputPort("pcd_stack"), f"{model_name}_pcd_stack"
                )

In [52]:
def CustomMakeManipulationStation(
    model_directives=None,
    filename=None,
    time_step=0.002,
    iiwa_prefix="iiwa",
    wsg_prefix="wsg",
    camera_prefix="camera",
    prefinalize_callback=None,
    package_xmls=[],
):
    """ """
    builder = DiagramBuilder()
    # asdsadsadsdasdasdas
    # Add (only) the iiwa, WSG, and cameras to the scene.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=time_step)
    parser = Parser(plant)
    for p in package_xmls:
        parser.package_map().AddPackageXml(p)
    AddPackagePaths(parser)
    if model_directives:
        directives = LoadModelDirectivesFromString(model_directives)
        ProcessModelDirectives(directives, parser)
    if filename:
        parser.AddAllModelsFromFile(filename)
    if prefinalize_callback:
        prefinalize_callback(plant)
        
    plant.mutable_gravity_field().set_gravity_vector([0, 0, -0.15])
    plant.Finalize()

    for i in range(plant.num_model_instances()):
        model_instance = ModelInstanceIndex(i)
        model_instance_name = plant.GetModelInstanceName(model_instance)

        if model_instance_name.startswith(iiwa_prefix):
            print("adding iiwa ", model_instance_name)
            num_iiwa_positions = plant.num_positions(model_instance)
            print(model_instance_name, " num_iiwa_positions:", num_iiwa_positions)
            # I need a PassThrough system so that I can export the input port.
            iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
            builder.ExportInput(
                iiwa_position.get_input_port(), model_instance_name + "_position"
            )
            builder.ExportOutput(
                iiwa_position.get_output_port(),
                model_instance_name + "_position_commanded",
            )

            # Export the iiwa "state" outputs.
            demux = builder.AddSystem(
                Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions)
            )
            print(
                "size: ",
                plant.get_state_output_port(model_instance).size(),
                " ",
                demux.get_input_port().size(),
            )

            print(
                "size: ",
                demux.get_output_port(0).size(),
                " ",
                demux.get_output_port(1).size(),
            )

            builder.Connect(
                plant.get_state_output_port(model_instance), demux.get_input_port()
            )
            builder.ExportOutput(
                demux.get_output_port(0), model_instance_name + "_position_measured"
            )
            builder.ExportOutput(
                demux.get_output_port(1), model_instance_name + "_velocity_estimated"
            )
            builder.ExportOutput(
                plant.get_state_output_port(model_instance),
                model_instance_name + "_state_estimated",
            )

            # Make the plant for the iiwa controller to use.
            controller_plant = MultibodyPlant(time_step=time_step)
            # # TODO: Add the correct IIWA model (introspected from MBP)
            # if plant.num_positions(model_instance) == 3:
            #     controller_iiwa = AddPlanarIiwa(controller_plant)
            # else:
            # controller_iiwa = AddIiwa(controller_plant)
            sdf_path = FindResourceOrThrow(
                "drake/manipulation/models/iiwa_description/iiwa7/"
                f"iiwa7_no_collision.sdf"
            )

            # AddIiwa {

            parser = Parser(controller_plant)

            controller_iiwa = parser.AddModelFromFile(
                "iiwa_description/iiwa7/iiwa7_with_box_collision.sdf"
            )
            # controller_iiwa = parser.AddModelFromFile("iiwa_description/iiwa7/iiwa7_no_collision.sdf")

            #  controller_iiwa = parser.AddModelFromFile("package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
            # controller_iiwa = parser.AddModelFromFile(sdf_path)

            controller_plant.WeldFrames(
                controller_plant.world_frame(),
                controller_plant.GetFrameByName("iiwa_link_0"),
            )
            # freebox_body_x = controller_plant.AddRigidBody(
            #         "freebox_body_x", iiwa,
            #         SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))

            # freebox_joint_x = controller_plant.AddJoint(PrismaticJoint(
            #         "freebox_joint_x", controller_plant.world_frame(), controller_plant.GetFrameByName("iiwa_link_0_x"),
            #         [1, 0, 0]))
            # plant.AddJointActuator("freebox_joint_x_actuator", freebox_joint_x)

            # Set default positions:
            q0 = [0, 0, 0, 0.0, 0.1, 0, -1.2, 0, 1.6, 0]
            index = 0
            for joint_index in controller_plant.GetJointIndices(controller_iiwa):
                joint = controller_plant.get_mutable_joint(joint_index)
                if isinstance(joint, RevoluteJoint):
                    joint.set_default_angle(q0[index])
                    index += 1
                if isinstance(joint, PrismaticJoint):
                    joint.set_default_translation(q0[index])
                    index += 1

            # } AddIiwa
            # add_freebox(controller_plant)

            AddWsg(controller_plant, controller_iiwa, welded=True)
            controller_plant.Finalize()

            kp = np.array([100] * num_iiwa_positions)
            print("num_iiwa_positions ", num_iiwa_positions)
            print("dim=kp.size() ", kp.size)
            print("controller_plant.num_positions ", controller_plant.num_positions())
            print("controller_plant.num_velocities ", controller_plant.num_velocities())
            print("controller_plant.num_actuators ", controller_plant.num_actuators())
            # Add the iiwa controller
            iiwa_controller = builder.AddSystem(
                InverseDynamicsController(
                    controller_plant,
                    kp=[100] * num_iiwa_positions,
                    ki=[1] * num_iiwa_positions,
                    kd=[20] * num_iiwa_positions,
                    has_reference_acceleration=False,
                )
            )
            iiwa_controller.set_name(model_instance_name + "_controller")
            builder.Connect(
                plant.get_state_output_port(model_instance),
                iiwa_controller.get_input_port_estimated_state(),
            )

            # Add in the feed-forward torque
            adder = builder.AddSystem(Adder(2, num_iiwa_positions))
            builder.Connect(
                iiwa_controller.get_output_port_control(), adder.get_input_port(0)
            )
            # Use a PassThrough to make the port optional (it will provide zero
            # values if not connected).
            torque_passthrough = builder.AddSystem(
                PassThrough([0] * num_iiwa_positions)
            )
            builder.Connect(
                torque_passthrough.get_output_port(), adder.get_input_port(1)
            )
            builder.ExportInput(
                torque_passthrough.get_input_port(),
                model_instance_name + "_feedforward_torque",
            )
            builder.Connect(
                adder.get_output_port(), plant.get_actuation_input_port(model_instance)
            )

            # Add discrete derivative to command velocities.
            desired_state_from_position = builder.AddSystem(
                StateInterpolatorWithDiscreteDerivative(
                    num_iiwa_positions, time_step, suppress_initial_transient=True
                )
            )
            desired_state_from_position.set_name(
                model_instance_name + "_desired_state_from_position"
            )
            builder.Connect(
                desired_state_from_position.get_output_port(),
                iiwa_controller.get_input_port_desired_state(),
            )
            builder.Connect(
                iiwa_position.get_output_port(),
                desired_state_from_position.get_input_port(),
            )

            # Export commanded torques.
            builder.ExportOutput(
                adder.get_output_port(), model_instance_name + "_torque_commanded"
            )
            builder.ExportOutput(
                adder.get_output_port(), model_instance_name + "_torque_measured"
            )

            builder.ExportOutput(
                plant.get_generalized_contact_forces_output_port(model_instance),
                model_instance_name + "_torque_external",
            )

        elif model_instance_name == "storage":
            print("adding storage")
            num_iiwa_positions = plant.num_positions(model_instance)
            print(model_instance_name, " num_iiwa_positions:", num_iiwa_positions)
            # I need a PassThrough system so that I can export the input port.
            iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
            builder.ExportInput(
                iiwa_position.get_input_port(), model_instance_name + "_position"
            )
            builder.ExportOutput(
                iiwa_position.get_output_port(),
                model_instance_name + "_position_commanded",
            )

            # Export the iiwa "state" outputs.
            demux = builder.AddSystem(
                Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions)
            )
            print(
                "size: ",
                plant.get_state_output_port(model_instance).size(),
                " ",
                demux.get_input_port().size(),
            )

            print(
                "size: ",
                demux.get_output_port(0).size(),
                " ",
                demux.get_output_port(1).size(),
            )

            builder.Connect(
                plant.get_state_output_port(model_instance), demux.get_input_port()
            )
            builder.ExportOutput(
                demux.get_output_port(0), model_instance_name + "_position_measured"
            )
            builder.ExportOutput(
                demux.get_output_port(1), model_instance_name + "_velocity_estimated"
            )
            builder.ExportOutput(
                plant.get_state_output_port(model_instance),
                model_instance_name + "_state_estimated",
            )

            # Make the plant for the iiwa controller to use.
            controller_plant = MultibodyPlant(time_step=time_step)
            # # TODO: Add the correct IIWA model (introspected from MBP)
            # if plant.num_positions(model_instance) == 3:
            #     controller_iiwa = AddPlanarIiwa(controller_plant)
            # else:
            # controller_iiwa = AddIiwa(controller_plant)

            # AddIiwa {

            parser = Parser(controller_plant)

            controller_iiwa = parser.AddModelFromFile(
                "iiwa_description/iiwa7/storage.sdf"
            )
            # controller_iiwa = parser.AddModelFromFile("iiwa_description/iiwa7/iiwa7_no_collision.sdf")

            #  controller_iiwa = parser.AddModelFromFile("package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
            # controller_iiwa = parser.AddModelFromFile(sdf_path)

            controller_plant.WeldFrames(
                controller_plant.world_frame(),
                controller_plant.GetFrameByName("iiwa_link_0"),
            )
            # freebox_body_x = controller_plant.AddRigidBody(
            #         "freebox_body_x", iiwa,
            #         SpatialInertia(0, [0, 0, 0], UnitInertia(0, 0, 0)))

            # freebox_joint_x = controller_plant.AddJoint(PrismaticJoint(
            #         "freebox_joint_x", controller_plant.world_frame(), controller_plant.GetFrameByName("iiwa_link_0_x"),
            #         [1, 0, 0]))
            # plant.AddJointActuator("freebox_joint_x_actuator", freebox_joint_x)

            # Set default positions:
            q0 = [5.0, 0.0, 0.0]
            index = 0
            for joint_index in controller_plant.GetJointIndices(controller_iiwa):
                joint = controller_plant.get_mutable_joint(joint_index)
                if isinstance(joint, RevoluteJoint):
                    joint.set_default_angle(q0[index])
                    index += 1
                if isinstance(joint, PrismaticJoint):
                    joint.set_default_translation(q0[index])
                    index += 1

            # } AddIiwa
            # add_freebox(controller_plant)

            CustomAddWsg(controller_plant, controller_iiwa, welded=True)
            controller_plant.Finalize()

            kp = np.array([100] * num_iiwa_positions)
            print("num_iiwa_positions ", num_iiwa_positions)
            print("dim=kp.size() ", kp.size)
            print("controller_plant.num_positions ", controller_plant.num_positions())
            print("controller_plant.num_velocities ", controller_plant.num_velocities())
            print("controller_plant.num_actuators ", controller_plant.num_actuators())

            # Add the iiwa controller
            iiwa_controller = builder.AddSystem(
                InverseDynamicsController(
                    controller_plant,
                    kp=[100] * num_iiwa_positions,
                    ki=[1] * num_iiwa_positions,
                    kd=[20] * num_iiwa_positions,
                    has_reference_acceleration=False,
                )
            )
            iiwa_controller.set_name(model_instance_name + "_controller")
            builder.Connect(
                plant.get_state_output_port(model_instance),
                iiwa_controller.get_input_port_estimated_state(),
            )

            # Add in the feed-forward torque
            adder = builder.AddSystem(Adder(2, num_iiwa_positions))
            builder.Connect(
                iiwa_controller.get_output_port_control(), adder.get_input_port(0)
            )
            # Use a PassThrough to make the port optional (it will provide zero
            # values if not connected).
            torque_passthrough = builder.AddSystem(
                PassThrough([0] * num_iiwa_positions)
            )
            builder.Connect(
                torque_passthrough.get_output_port(), adder.get_input_port(1)
            )
            builder.ExportInput(
                torque_passthrough.get_input_port(),
                model_instance_name + "_feedforward_torque",
            )
            builder.Connect(
                adder.get_output_port(), plant.get_actuation_input_port(model_instance)
            )

            # Add discrete derivative to command velocities.
            desired_state_from_position = builder.AddSystem(
                StateInterpolatorWithDiscreteDerivative(
                    num_iiwa_positions, time_step, suppress_initial_transient=True
                )
            )
            desired_state_from_position.set_name(
                model_instance_name + "_desired_state_from_position"
            )
            builder.Connect(
                desired_state_from_position.get_output_port(),
                iiwa_controller.get_input_port_desired_state(),
            )
            builder.Connect(
                iiwa_position.get_output_port(),
                desired_state_from_position.get_input_port(),
            )

            # Export commanded torques.
            builder.ExportOutput(
                adder.get_output_port(), model_instance_name + "_torque_commanded"
            )
            builder.ExportOutput(
                adder.get_output_port(), model_instance_name + "_torque_measured"
            )

            builder.ExportOutput(
                plant.get_generalized_contact_forces_output_port(model_instance),
                model_instance_name + "_torque_external",
            )
        elif model_instance_name.startswith(wsg_prefix):
            # Wsg controller.
            wsg_controller = builder.AddSystem(SchunkWsgPositionController())
            wsg_controller.set_name(model_instance_name + "_controller")
            builder.Connect(
                wsg_controller.get_generalized_force_output_port(),
                plant.get_actuation_input_port(model_instance),
            )
            builder.Connect(
                plant.get_state_output_port(model_instance),
                wsg_controller.get_state_input_port(),
            )
            builder.ExportInput(
                wsg_controller.get_desired_position_input_port(),
                model_instance_name + "_position",
            )
            builder.ExportInput(
                wsg_controller.get_force_limit_input_port(),
                model_instance_name + "_force_limit",
            )
            wsg_mbp_state_to_wsg_state = builder.AddSystem(
                MakeMultibodyStateToWsgStateSystem()
            )
            builder.Connect(
                plant.get_state_output_port(model_instance),
                wsg_mbp_state_to_wsg_state.get_input_port(),
            )
            builder.ExportOutput(
                wsg_mbp_state_to_wsg_state.get_output_port(),
                model_instance_name + "_state_measured",
            )
            builder.ExportOutput(
                wsg_controller.get_grip_force_output_port(),
                model_instance_name + "_force_measured",
            )

    # Cameras.
    CustomAddRgbdSensors(
        builder, plant, scene_graph, model_instance_prefix=camera_prefix
    )

    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "query_object")
    builder.ExportOutput(plant.get_contact_results_output_port(), "contact_results")
    builder.ExportOutput(plant.get_state_output_port(), "plant_continuous_state")
    builder.ExportOutput(plant.get_body_poses_output_port(), "body_poses")

    diagram = builder.Build()
    diagram.set_name("ManipulationStation")
    return diagram


saat()

2023-05-31 16:14:21.229359 


# CustomAddIiwaDifferentialIK

In [53]:
params = 0


def CustomAddIiwaDifferentialIK(builder, plant, frame=None, n=0, position_limits=None):
    global params
    params = DifferentialInverseKinematicsParameters(
        plant.num_positions(), plant.num_velocities()
    )
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(2)
    # params.set_end_effector_translational_velocity_limits([-2, -2, -2],
    #                                                       [2, 2, 2])
    if plant.num_positions() == 3:  # planar iiwa
        frame = plant.GetFrameByName("iiwa_link_0_z")
        iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits)
        )
        # These constants are in body frame
        # assert (
        #     frame.name() == "iiwa_link_7"
        # ), "Still need to generalize the remaining planar diff IK params for different frames"  # noqa
        # params.set_end_effector_velocity_flag(
        #     [True, False, False, True, False, True])
    else:
        iiwa14_velocity_limits = np.array(
            [2, 2, 2, 1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3]
        )
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits)
        )

        # matrix = 50 * np.eye(10)

        # for i in range(3):
        #     for k in range(10):
        #         matrix[i, k] = 0.1
        # params.set_joint_centering_gain(matrix)

    if frame is None:
        frame = plant.GetFrameByName("body")

    print("frame", frame)
    differential_ik = builder.AddSystem(
        DifferentialInverseKinematicsIntegrator(
            plant, frame, time_step, params, log_only_when_result_state_changes=True
        )
    )

    return differential_ik

# ICP

In [54]:
# Takes 3 point clouds (in world coordinates) as input, and outputs and estimated pose for the mustard bottle.


class MustardIterativeClosestPoint(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        model_point_cloud = AbstractValue.Make(PointCloud(0))

        self.inputs = []
        self.noOfCams = 2

        img1 = AbstractValue.Make(ImageRgba8U(480, 640, 0))
        depth1 = AbstractValue.Make(ImageDepth32F(480, 640, 0.0))



        # deneme amacli port
        # self.DeclareAbstractInputPort("cloud3", AbstractValue.Make([np.ndarray]))
        self.DeclareAbstractInputPort("cam4_point_cloud", AbstractValue.Make([np.ndarray]))
        self.DeclareAbstractInputPort("cam5_point_cloud", AbstractValue.Make([np.ndarray]))

        self.DeclareAbstractOutputPort(
            "X_WO", lambda: AbstractValue.Make([RigidTransform()]), self.EstimatePose
        )

        self.box = CrackerBoxPointCloud.CrackerBoxPointCloud()
        # meshcat.SetObject("icp_scene", self.box)
        # meshcat.SetObject("icp_model", self.box)

    def EstimatePose(self, context, output):
        pcd = []
        print("EstimatePose")
        # global inputs

        
            # inputs[i] = (self.get_input_port(i).Eval(context))

        for i in range(self.noOfCams):
            self.inputs.append(self.get_input_port(i).Eval(context))


        # for i in range(len(self.inputs[9][0])):
        #     pcd.append(self.inputs[9][0][i])
        # print("pcd", pcd)
        pose_estimates = []

        for i in range(len(self.inputs)):
            if isinstance(self.inputs[i][0][0], int):
                print("inputs[i][0] = [-1] !!!!! i: ", i)
                continue
            noOfObjects = len(self.inputs[i][0])
            print("noOfObjects: ", noOfObjects)
            for k in range(noOfObjects):
                pcd = [self.inputs[i][0][k]]

                merged_pcd = Concatenate(pcd)
                down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
                meshcat.SetObject("icp_observations", down_sampled_pcd, point_size=0.001)

                # takes icp_observations + model cloud , returns icp?s
                X_WOhat, chat = IterativeClosestPoint(
                    self.box.xyzs(),
                    down_sampled_pcd.xyzs(),
                    meshcat=meshcat,
                    meshcat_scene_path="icp_scene" + str(k) + "from cam: " + str(i),
                )
                pose_estimates.append(X_WOhat)
                print("X_WOhat = {} for ".format(X_WOhat) + "index" + str(k) + "from cam: " + str(i))

        output.set_value(pose_estimates)

In [55]:
saat("before def icp")
# def icp_pick_and_place_demo():
builder = DiagramBuilder()

station = builder.AddSystem(
    CustomMakeManipulationStation(
        # model_directives, prefinalize_callback=setup
        model_directives, time_step=0.002, prefinalize_callback=setup
    )
)
print("after add station")


plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# iiwa = plant.GetModelInstanceByName("iiwa2")
# print("num pos iiwa2",plant.num_positions(iiwa))


# Find camera frames
frames = []

frames.append(plant.GetFrameByName(f"camera{4}_origin"))
AddMeshcatTriad(
        meshcat, f"camera{4}_origin", X_PT=frames[0].GetFixedPoseInBodyFrame()
    )
# for each frame add triads
# for i in range(1):
#     AddMultibodyTriad(frames[i], scene_graph, length=.1, radius=0.005)

# Add meshcet triads for cameras

# for i in range(3):
#     AddMeshcatTriad(
#         meshcat, f"camera{i+3}_origin", X_PT=frames[i].GetFixedPoseInBodyFrame()
#     )


icp = builder.AddSystem((MustardIterativeClosestPoint()))

# deneme
# builder.Connect(station.GetOutputPort("camera3_masked_depth_image"),
#                 icp.get_input_port(9))

builder.Connect(station.GetOutputPort("camera4_pcd_stack"), icp.get_input_port(0))
builder.Connect(station.GetOutputPort("camera5_pcd_stack"), icp.get_input_port(1))


plan = builder.AddSystem(PickAndPlaceTrajectory3(plant))
builder.Connect(station.GetOutputPort("body_poses"), plan.GetInputPort("body_poses"))
builder.Connect(icp.GetOutputPort("X_WO"), plan.GetInputPort("X_WO"))

robot = station.GetSubsystemByName("iiwa_controller").get_multibody_plant_for_control()

# Set up differential inverse kinematics.
diff_ik = CustomAddIiwaDifferentialIK(builder, robot, None, 0)
builder.Connect(diff_ik.get_output_port(), station.GetInputPort("iiwa_position"))
builder.Connect(plan.GetOutputPort("X_WG"), diff_ik.get_input_port(0))
builder.Connect(
    station.GetOutputPort("iiwa_state_estimated"), diff_ik.GetInputPort("robot_state")
)


builder.Connect(
    plan.GetOutputPort("wsg_position"), station.GetInputPort("wsg_position")
)
# --------------------------------- add storage controller and differential ik

robot2 = station.GetSubsystemByName(
    "storage_controller"
).get_multibody_plant_for_control()
# ---------------------------------
# connect storage planner
storage_planner = builder.AddSystem(StoragePlanner(plant))

builder.Connect(
    station.GetOutputPort("body_poses"), storage_planner.GetInputPort("body_poses")
)

diff_ik2 = CustomAddIiwaDifferentialIK(builder, robot2, None, 2)
builder.Connect(storage_planner.GetOutputPort("X_WG"), diff_ik2.get_input_port(0))

builder.Connect(
    station.GetOutputPort("storage_state_estimated"),
    diff_ik2.GetInputPort("robot_state"),
)

builder.Connect(diff_ik2.get_output_port(), station.GetInputPort("storage_position"))

# --------------------------------- add iiwa2 controller and differential ik

robot3 = station.GetSubsystemByName("iiwa2_controller").get_multibody_plant_for_control()


diff_ik_iiwa2 = CustomAddIiwaDifferentialIK(builder, robot3, None, 0)
builder.Connect(diff_ik_iiwa2.get_output_port(), station.GetInputPort("iiwa2_position"))
builder.Connect(plan.GetOutputPort("X_WG2"), diff_ik_iiwa2.get_input_port(0))
builder.Connect(
    station.GetOutputPort("iiwa2_state_estimated"), diff_ik_iiwa2.GetInputPort("robot_state")
)


builder.Connect(
    plan.GetOutputPort("wsg_position2"), station.GetInputPort("wsg2_position")
)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, station.GetOutputPort("query_object"), meshcat
)


global diagram
diagram = builder.Build()

simulator = Simulator(diagram)
context = simulator.get_context()
# print("camera3_pcd_stack", station.GetOutputPort('camera3_pcd_stack').Eval(station.GetSubsystemByName("CreatePointclouds").GetMyContextFromRoot(context)))

simulator.Initialize()

# color_image = station.GetOutputPort("camera3_point_cloud").Eval(context)
# depth_image = station.GetOutputPort("camera3_point_cloud").Eval(context)

# # Plot the two images.
# plt.subplot(121)
# plt.imshow(color_image.data)
# plt.title('Color image')
# plt.subplot(122)
# plt.imshow(np.squeeze(depth_image.data))
# plt.title('Depth image')
# mpld3.display()
# plt.show()


if True:  # draw the trajectory triads
    X_G_traj = plan.GetMyContextFromRoot(context).get_abstract_state(
        0).get_value()
    # visualize_trajectory(X_G_traj, visualizer)
    i = 0
    for t in np.linspace(X_G_traj.start_time(), X_G_traj.end_time(), 70):
        i = i + 1
        AddMeshcatTriad(meshcat,
                        f"X_G/({chr(97 + i - 1)})",
                        X_PT=X_G_traj.GetPose(t),
                        length=.1,
                        radius=0.003)
        
    X_G2_traj = plan.GetMyContextFromRoot(context).get_abstract_state(
        2).get_value()
    # visualize_trajectory(X_G_traj, visualizer)
    i = 0
    for t in np.linspace(X_G2_traj.start_time(), X_G2_traj.end_time(), 70):
        i = i + 1
        AddMeshcatTriad(meshcat,
                        f"X_G2/({chr(97 + i - 1)})",
                        X_PT=X_G2_traj.GetPose(t),
                        length=.1,
                        radius=0.004)
        
    X_W_S_traj = (
        storage_planner.GetMyContextFromRoot(context).get_abstract_state(0).get_value()
    )
    i = 0
    for t in np.linspace(X_W_S_traj.start_time(), X_W_S_traj.end_time(), 70):
        i = i + 1
        AddMeshcatTriad(
            meshcat,
            f"X_Storage/({chr(97 + i - 1)})",
            X_PT=X_W_S_traj.GetPose(t),
            length=0.1,
            radius=0.005,
        )
print("end time: ", plan.end_time(plan.GetMyContextFromRoot(context)))
if running_as_notebook:
    visualizer.StartRecording(False)
    simulator.AdvanceTo(plan.end_time(plan.GetMyContextFromRoot(context)))
    # simulator.AdvanceTo(0.1)
    
    visualizer.PublishRecording()
else:
    simulator.AdvanceTo(0.1)

saat("before calling icp")
# icp_pick_and_place_demo()
saat("after calling icp")

2023-05-31 16:14:21.505511 before def icp
adding iiwa  iiwa
iiwa  num_iiwa_positions: 10
size:  20   20
size:  10   10
num_iiwa_positions  10
dim=kp.size()  10
controller_plant.num_positions  10
controller_plant.num_velocities  10
controller_plant.num_actuators  10
adding storage
storage  num_iiwa_positions: 3
size:  6   6
size:  3   3
num_iiwa_positions  3
dim=kp.size()  3
controller_plant.num_positions  3
controller_plant.num_velocities  3
controller_plant.num_actuators  3
adding iiwa  iiwa2
iiwa2  num_iiwa_positions: 10
size:  20   20
size:  10   10
num_iiwa_positions  10
dim=kp.size()  10
controller_plant.num_positions  10
controller_plant.num_velocities  10
controller_plant.num_actuators  10
after add station
frame <BodyFrame_[float] name='body' index=30 model_instance=3>
frame <BodyFrame_[float] name='iiwa_link_0_z' index=3 model_instance=2>
frame <BodyFrame_[float] name='body' index=30 model_instance=3>
EstimatePose


KeyboardInterrupt: 

: 

# error

In [ ]:
with open("dg.svg", "wb") as output_file:
    output_file.write(
        pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=2))[
            0
        ].create_svg()
    )

In [ ]:
plt.imshow(np.squeeze(icp.inputs[8].data))
plt.title("Depth image")

IndexError: list index out of range

: 

: 

: 

In [ ]:
print(icp.inputs[9])

In [ ]:
# import matplotlib.pyplot as plt
# plt.imshow(np.squeeze(icp.inputs[5].data))
# plt.title('Depth image')

# res = icp.inputs[5].data

# #and masks[i]['area'] > 30000
# masks = mask_generator.generate(res[:,:,:3])
# for i in range(len(masks)):
#     if (masks[i]['stability_score'] > 0.99  ):
#         image = np.array(masks[i]["segmentation"])
#         plt.figure(figsize=(20,20))
#         plt.imshow(image)
#         plt.axis('off')
#         plt.show()